<h3>Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.</h3>

<h3>Q2: How many facilities do not charge a fee to members?</h3>

4

<h3>Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.</h3>

<h3>Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.</h3>

In [ ]:
SELECT *
FROM Facilities 
WHERE facid in (1,5)

<h3>Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.</h3>

<h3> Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. </h3>

<h3> Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.</h3>

Can't get this to work properly. doesn't seem to let me concatenate text (I keep getting a 403 error). Here is what I have tried.

<h3> Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.</h3>

In [ ]:
SELECT A.name AS Facility, C.surname, C.firstname,
CASE WHEN B.memid = 0 THEN A.guestcost * B.slots
Else A.membercost * B.slots
END AS Cost
FROM Facilities AS A
JOIN Bookings AS B
ON A.facid = B.facid
JOIN Members AS C
ON B.memid = C.memid
WHERE B.starttime >= '2012-09-14'
AND B.starttime < '2012-09-15'
HAVING Cost > 30.0
ORDER BY Cost DESC

<h3> Q9: This time, produce the same result as in Q8, but using a subquery. </h3>

getting a 403 error again

In [16]:
import sqlite3
import pandas as pd

In [4]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
#TABLES IN DB
#Facilities
#Members
#Bookings

In [5]:
cur = con.cursor()

In [17]:
Facilities = pd.read_sql_query("SELECT * from Facilities", con)
Bookings = pd.read_sql_query("SELECT * from Bookings", con)
Members = pd.read_sql_query("SELECT * from Members", con)

<h3> Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! </h3>

In [35]:
BF_df = Bookings.join(Facilities, on = 'facid', lsuffix = 'L')
BF_df['revenue'] = BF_df['membercost']*BF_df['slots']*(BF_df['memid']!=0) + BF_df['guestcost']*BF_df['slots']*(BF_df['memid']==0)
BF_df_rev = BF_df[['revenue','name']].groupby('name').sum()
BF_df_rev[BF_df_rev['revenue']<1000].sort_values('revenue', ascending = False)

revenue
name                  
Pool Table       270.0
Snooker Table    240.0
Table Tennis     180.0

<h3> Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order </h3>

In [56]:
members_reccomended = Members[Members['recommendedby'].str.isnumeric()].copy()
members_reccomended['recommendedby'] = members_reccomended['recommendedby'].astype('int64')
members_reccomended[['memid','surname','firstname','recommendedby']].merge(
    Members[['memid','surname','firstname']], how = 'left', left_on = 'recommendedby', right_on = 'memid',
    suffixes=('', '_recommender'))[['surname', 'firstname', 'surname_recommender', 'firstname_recommender']].sort_values(['surname', 'firstname'])

surname  firstname surname_recommender firstname_recommender
9               Bader   Florence            Stibbons                Ponder
7               Baker       Anne            Stibbons                Ponder
10              Baker    Timothy             Farrell                Jemima
3              Boothe        Tim              Rownam                   Tim
1             Butters     Gerald               Smith                Darren
14             Coplin       Joan               Baker               Timothy
21            Crumpet      Erica               Smith                 Tracy
2                Dare      Nancy            Joplette                Janice
12            Genting    Matthew             Butters                Gerald
20               Hunt       John             Purview             Millicent
6               Jones      David            Joplette                Janice
16              Jones    Douglas               Jones                 David
0            Joplette     Janice               Smith                Darren
13          Mackenzie       Anna               Smith                Darren
5                Owen    Charles               Smith                Darren
11             Pinker      David             Farrell                Jemima
19            Purview  Millicent               Smith                 Tracy
17             Rumney  Henrietta             Genting               Matthew
15             Sarwin  Ramnaresh               Bader              Florence
8               Smith       Jack               Smith                Darren
4            Stibbons     Ponder               Tracy                Burton
18  Worthington-Smyth      Henry               Smith                 Tracy

<h3> Q12: Find the facilities with their usage by member, but not guests </h3>

In [64]:
BF_df = Bookings.merge(Facilities, on = 'facid')[['memid','name','slots']]
BF_df[BF_df['memid'] != 0].groupby(['name','memid']).sum()

slots
name            memid       
Badminton Court 1        432
                2        102
                3         12
                5         63
                6          6
...                      ...
Tennis Court 2  24        36
                27         3
                28         3
                30         3
                35        12

[202 rows x 1 columns]

In [70]:
BF_df = Bookings.merge(Facilities, on = 'facid')[['memid','name','slots']]
BF_df[BF_df['memid'] != 0].groupby(['name']).sum().drop(columns = 'memid').sort_values('slots')

slots
name                  
Massage Room 2      54
Squash Court       418
Table Tennis       794
Pool Table         856
Snooker Table      860
Tennis Court 2     882
Massage Room 1     884
Tennis Court 1     957
Badminton Court   1086

<h3> Q13: Find the facilities usage by month, but not guests </h3>

In [90]:
BF_df = Bookings.merge(Facilities, on = 'facid')[['starttime','slots','name','memid']]
BF_df['month'] = pd.to_datetime(BF_df['starttime'], format = "%Y-%m-%d").apply(lambda x : x.month)
BF_df[BF_df['memid'] != 0][['name','month','slots']].groupby(['name','month']).sum()

slots
name            month       
Badminton Court 7        165
                8        414
                9        507
Massage Room 1  7        166
                8        316
                9        402
Massage Room 2  7          8
                8         18
                9         28
Pool Table      7        110
                8        303
                9        443
Snooker Table   7        140
                8        316
                9        404
Squash Court    7         50
                8        184
                9        184
Table Tennis    7         98
                8        296
                9        400
Tennis Court 1  7        201
                8        339
                9        417
Tennis Court 2  7        123
                8        345
                9        414